In [6]:
import pandas as pd
import json
import numpy as np
import re
p = re.compile('(?<!\\\\)\'')
# s = p.sub('\"', s)

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [7]:
# change the input file and the output filename for different cities
rv_sl = pd.read_csv('calendar_salem.csv')
rv_sl.dtypes

listing_id         int64
date              object
available         object
price             object
adjusted_price    object
minimum_nights     int64
maximum_nights     int64
dtype: object

In [8]:
rv_sl.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,12899,2022-12-23,f,$80.00,$80.00,3,1125
1,12899,2022-12-24,f,$80.00,$80.00,3,1125
2,12899,2022-12-25,f,$80.00,$80.00,3,1125
3,12899,2022-12-26,f,$80.00,$80.00,3,1125
4,12899,2022-12-27,f,$80.00,$80.00,3,1125


In [9]:
dic_rv_sl = {}
types = {}
for i in range(len(rv_sl.columns)):
#     print(lst_sl.columns[i], lst_sl.dtypes[i]=='float64')
    currcol = rv_sl.columns[i]
    if currcol in ['price', 'adjusted_price']:
        types[currcol] = float
        continue
    currtype = rv_sl.dtypes[i]
    newtype = ''
    if currtype=='int64':
        newtype = int
    elif currtype==object:
        newtype = str
    elif currtype=='float64':
        newtype = float
    types[currcol] = newtype
    
final = {}
final_arr = []
for j in range(rv_sl.shape[0]):
    temp = {}
    for col in rv_sl.columns:
        currcontent = rv_sl.iloc[j][col]
        if (col=='price' or col=='adjusted_price'):
            temp[col] = float(currcontent.replace('$','').replace(',',''))
        else:
            temp[col] = types[col](currcontent)
    final_arr.append(temp)
final["calendar_info"] = final_arr
    
with open("calendar_sl.json", "w") as file:
    json.dump(final, file, indent=4, sort_keys=True,
              separators=(', ', ': '),
              cls=NpEncoder)